In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import seaborn as sns
from pandas_profiling import ProfileReport
import plotly.express as px

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
life_df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/life-expectancy-income.csv')
lite_df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/literacy-rates.csv')

In [0]:
lite_df['District']=lite_df['District'].str.rstrip()

In [5]:
set(life_df['District'].unique()).difference(lite_df['District'].unique())

{'Chitwan', 'Tehrathum'}

In [0]:
lite_df['District']=lite_df['District'].map(lambda x: 'Chitwan' if x=='Chitawan' else x)
lite_df['District']=lite_df['District'].map(lambda x: 'Tehrathum' if x=='Terhathum' else x)

In [0]:
lite_df.drop([' Year'],axis=1,inplace=True)

In [0]:
mixed_df = pd.merge(lite_df,life_df,on='District',how='inner')
mixed_df.columns=mixed_df.columns.str.strip()

In [9]:
mixed_df.columns

Index(['District', 'Total', 'Female', 'Male', 'Life expectancy(In Years)',
       'Per Capita Income(In USD)'],
      dtype='object')

## **Standarization**

In [0]:
from sklearn.preprocessing import StandardScaler

def scaling(data,column):
  scaler=StandardScaler()
  X=data.drop(columns=[column])
  scaler.fit(X)
  scaled_data=scaler.transform(X)
  return scaled_data,X

scaled_merged,X_merged=scaling(mixed_df.drop(columns=['Total']),'District')
scaled_life,X_life=scaling(life_df,'District')
scaled_lite,X_lite=scaling(lite_df.drop(columns=['Total']),'District')

In [11]:
print('Scaled Merged Dataset: ',scaled_merged.shape)
print('Scaled Life E Dataset: ',scaled_life.shape)
print('Scaled Litera Dataset: ',scaled_lite.shape)

Scaled Merged Dataset:  (75, 4)
Scaled Life E Dataset:  (75, 2)
Scaled Litera Dataset:  (75, 2)


## **PCA**

In [0]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3, random_state=1)
pca =pca.fit(scaled_merged)
scaled_transformed=pca.transform(scaled_merged)

scaled_transformed_df=pd.DataFrame(scaled_transformed)
scaled_transformed_df.columns=['Feature 1','Feature 2','Feature 3']

colors = ['deeppink', 'maroon', 'green', 'yellow', 'skyblue', 'purple',  'red', 'black']
vectorizer = np.vectorize(lambda x: colors[x % len(colors)])

In [0]:
plot_data=pd.DataFrame(scaled_transformed, columns=['Feature 1','Feature 2','Feature 3'])
plot_data['District'] = mixed_df['District'].tolist()

In [21]:
plot_data

,Feature 1,Feature 2,Feature 3,District
0,4.337676,0.680265,2.022282,Kathmandu
1,3.118809,-0.340032,0.521001,Lalitpur
2,2.830949,-0.513740,-0.160116,Kaski
3,2.516811,-0.536817,-0.480155,Bhaktapur
4,1.729114,0.420275,-0.447127,Ilam
...,...,...,...,...
70,-2.118363,-1.613603,0.263249,Siraha
71,-2.829681,1.547524,0.507954,Humla
72,-2.780561,-0.920728,0.527663,Mahottari
73,-2.667351,-1.163705,0.829819,Sarlahi


In [23]:
fig=px.scatter(plot_data, x= 'Feature 1',y='Feature 2',hover_data=['District'],width=800, height=600)
fig.update_layout(
    title="Plot of HDI Dataset",
    xaxis_title="Feature 1",
    yaxis_title="Feature 2",
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="#7f7f7f"
    )
)
fig.show()

In [0]:
from sklearn import svm
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

## **Elliptic Envelope**

In [0]:
plt_data=plot_data.iloc[:,0:2]

elip_env = EllipticEnvelope(random_state=0)
elip_env.fit(plt_data)
y_pred=elip_env.predict(plt_data)
# predict returns 1 for an inlier and -1 for an outlier

In [60]:
y_pred

array([-1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1, -1,  1,  1,  1, -1,  1,  1, -1,  1, -1,  1,  1,
        1,  1,  1, -1,  1,  1, -1])

In [61]:
plot_data

,Feature 1,Feature 2,Feature 3,District,Elliptic,Elliptic3D
0,4.337676,0.680265,2.022282,Kathmandu,red,-1
1,3.118809,-0.340032,0.521001,Lalitpur,green,1
2,2.830949,-0.513740,-0.160116,Kaski,green,1
3,2.516811,-0.536817,-0.480155,Bhaktapur,green,1
4,1.729114,0.420275,-0.447127,Ilam,green,1
...,...,...,...,...,...,...
70,-2.118363,-1.613603,0.263249,Siraha,green,1
71,-2.829681,1.547524,0.507954,Humla,green,1
72,-2.780561,-0.920728,0.527663,Mahottari,green,1
73,-2.667351,-1.163705,0.829819,Sarlahi,green,1


In [62]:
fig=px.scatter(plot_data, x= 'Feature 1',y='Feature 2',color=y_pred,hover_data=['District'],width=800, height=600)
fig.update_layout(
    title="Outliers by Eliptic Envelope on Agriculture Dataset",
    xaxis_title="Feature 1",
    yaxis_title="Feature 2",
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="#7f7f7f"
    )
)
fig.show()

In [63]:
plt_data=plot_data.iloc[:,0:3]

elip_env = EllipticEnvelope(random_state=0)
elip_env.fit(plt_data)
y_pred=elip_env.predict(plt_data)
# plot_data['Elliptic3D']=y_pred


fig=px.scatter_3d(plot_data, x= 'Feature 1',y='Feature 2',z='Feature 3',color=y_pred,hover_data=['District'],width=900, height=600,size_max=10,opacity=0.7)
fig.update_layout(
    title="Outlier detection 3D Visualization on Agriculture Dataset ",
    xaxis_title="Feature 1",
    yaxis_title="Feature 2",
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="#7f7f7f"
    )
)
fig.show()

## **One Class SVM**

In [0]:
one_class=svm.OneClassSVM(kernel='rbf',gamma='auto')
plt_data2d=plot_data.iloc[:,0:2]
one_class.fit(plt_data2d)
y_pred2d=one_class.predict(plt_data2d)

one_class=svm.OneClassSVM(kernel='rbf',gamma='auto')
plt_data3d=plot_data.iloc[:,0:3]
one_class.fit(plt_data3d)
y_pred3d=one_class.predict(plt_data3d)

In [69]:
# colors=np.array(['red','green'])
fig=px.scatter(plot_data, x= 'Feature 1',y='Feature 2',color=y_pred2d,hover_data=['District'],width=800, height=600)
fig.update_layout(
    title="Outliers by One Class SVM on HDI Dataset",
    xaxis_title="Feature 1",
    yaxis_title="Feature 2",
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="#7f7f7f"
    )
)
fig.show()

fig=px.scatter_3d(plot_data, x= 'Feature 1',y='Feature 2',z='Feature 3',color=y_pred3d,hover_data=['District'],width=900, height=600)
fig.update_layout(
    title="OneClass SVM Outlier detection 3D Visualization on HDI Dataset ",
    xaxis_title="Feature 1",
    yaxis_title="Feature 2",
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="#7f7f7f"
    )
)
fig.show()

## **Isolation Forest**

In [0]:
iso_forest=IsolationForest(random_state=0)
plt_data2d=plot_data.iloc[:,0:2]
iso_forest.fit(plt_data2d)
y_pred2d=iso_forest.predict(plt_data2d)

iso_forest=IsolationForest(random_state=0)
plt_data3d=plot_data.iloc[:,0:3]
iso_forest.fit(plt_data3d)
y_pred3d=iso_forest.predict(plt_data3d)

In [68]:
colors=np.array(['red','green'])
fig=px.scatter(plot_data, x= 'Feature 1',y='Feature 2',color=y_pred2d,hover_data=['District'],width=800, height=600)
fig.update_layout(
    title="Outliers by Isolation Forest on HDI Dataset",
    xaxis_title="Feature 1",
    yaxis_title="Feature 2",
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="#7f7f7f"
    )
)
fig.show()

fig=px.scatter_3d(plot_data, x= 'Feature 1',y='Feature 2',z='Feature 3',color=y_pred3d,hover_data=['District'],width=900, height=600)
fig.update_layout(
    title="Isolation Forest Outlier detection 3D Visualization on Agriculture Dataset ",
    xaxis_title="Feature 1",
    yaxis_title="Feature 2",
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="#7f7f7f"
    )
)
fig.show()


## **Local Outlier**

In [0]:
loc_outlier=LocalOutlierFactor()
plt_data2d=plot_data.iloc[:,0:2]
y_pred2d=loc_outlier.fit_predict(plt_data2d)


loc_outlier=LocalOutlierFactor()
plt_data3d=plot_data.iloc[:,0:3]
y_pred3d=loc_outlier.fit_predict(plt_data3d)

In [74]:
colors=np.array(['red','green'])
fig=px.scatter(plot_data, x= 'Feature 1',y='Feature 2',color=y_pred2d,hover_data=['District'],width=800, height=600)
fig.update_layout(
    title="Outliers by Local Outlier on HDI Dataset",
    xaxis_title="Feature 1",
    yaxis_title="Feature 2",
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="#7f7f7f"
    )
)
fig.show()

fig=px.scatter_3d(plot_data, x= 'Feature 1',y='Feature 2',z='Feature 3',color=y_pred3d,hover_data=['District'],width=900, height=600)
fig.update_layout(
    title="Local Outlier detection 3D Visualization on HDI Dataset ",
    xaxis_title="Feature 1",
    yaxis_title="Feature 2",
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="#7f7f7f"
    )
)
fig.show()
